### Create a database with 3 demo tables

In [0]:
db = "demo_subledger_monitoring"

# Setup database for the project
spark.sql(f"CREATE DATABASE IF NOT EXISTS {db}")
spark.sql(f"USE {db}")

# Raw transactions
spark.sql("""
    CREATE TABLE IF NOT EXISTS transactions_raw (
    transaction_id STRING,
    posting_date DATE,
    account_id STRING,
    counterparty STRING,
    currency STRING,
    amount DECIMAL(18,2),
    ingestion_ts TIMESTAMP
    )
    USING DELTA
""")

# Daily subledger balances
spark.sql("""
  CREATE TABLE IF NOT EXISTS subledger_daily (
    posting_date DATE,
    account_id STRING,
    currency STRING,
    total_amount DECIMAL(18,2),
    transaction_count BIGINT,
    computed_ts TIMESTAMP
  )
  USING DELTA
""")

# Expected daily totals
spark.sql("""
    CREATE TABLE IF NOT EXISTS expected_daily_totals (
        posting_date DATE,
        account_id STRING,
        currency STRING,
        expected_total DECIMAL(18,2),
        source STRING
    )
    USING DELTA
""")

# Incident log
spark.sql("""
  CREATE TABLE IF NOT EXISTS incident_log (
    incident_id STRING,
    incident_ts TIMESTAMP,
    severity STRING,
    category STRING,
    check_name STRING,
    posting_date DATE,
    account_id STRING,
    currency STRING,
    metric_name STRING,
    metric_value STRING,
    expected_value STRING,
    details STRING,
    status STRING
  )
  USING DELTA
""")

print(f"Created database and tables.")


Created database and tables.


### Insert sample data

In [0]:
spark.sql("USE demo_subledger_monitoring")

# Clean table data for repeatable demo
spark.sql("TRUNCATE TABLE transactions_raw")
spark.sql("TRUNCATE TABLE expected_daily_totals")

# Insert sample data with intentional errors
spark.sql("""
    INSERT INTO transactions_raw VALUES
    ('TXN-1001', DATE('2026-01-10'), '4000', 'Vendor A', 'EUR',  120.50, current_timestamp()),
    ('TXN-1002', DATE('2026-01-10'), '4000', 'Vendor B', 'EUR',   79.50, current_timestamp()),
    ('TXN-1003', DATE('2026-01-10'), '4010', 'Vendor C', 'EUR',  -15.00, current_timestamp()),   -- negative number
    (NULL,       DATE('2026-01-10'), '4010', 'Vendor D', 'EUR',   20.00, current_timestamp()),   -- missing id
    ('TXN-2001', DATE('2026-01-11'), '4000', 'Vendor A', 'EUR',   50.00, current_timestamp()),
    ('TXN-2002', DATE('2026-01-11'), '4000', 'Vendor A', 'EUR',   50.00, current_timestamp()),
    ('TXN-2003', DATE('2026-01-11'), '4020', 'Vendor E', 'USD',  100.00, current_timestamp())
""")

display(spark.sql("SELECT * FROM transactions_raw ORDER BY posting_date, account_id, transaction_id"))

# Intentionally create one mismatch
spark.sql("""
    INSERT INTO expected_daily_totals VALUES
        (DATE('2026-01-10'), '4000', 'EUR', 200.00, 'mock_gl'),
        (DATE('2026-01-10'), '4010', 'EUR',   6.00, 'mock_gl'),   -- mismatch on purpose
        (DATE('2026-01-11'), '4000', 'EUR', 100.00, 'mock_gl'),
        (DATE('2026-01-11'), '4020', 'USD', 100.00, 'mock_gl')
""")

display(spark.sql("SELECT * FROM expected_daily_totals ORDER BY posting_date, account_id, currency"))


transaction_id,posting_date,account_id,counterparty,currency,amount,ingestion_ts
TXN-1001,2026-01-10,4000,Vendor A,EUR,120.50,2026-01-11T21:04:10.393Z
TXN-1002,2026-01-10,4000,Vendor B,EUR,79.50,2026-01-11T21:04:10.393Z
null,2026-01-10,4010,Vendor D,EUR,20.00,2026-01-11T21:04:10.393Z
TXN-1003,2026-01-10,4010,Vendor C,EUR,-15.00,2026-01-11T21:04:10.393Z
TXN-2001,2026-01-11,4000,Vendor A,EUR,50.00,2026-01-11T21:04:10.393Z
TXN-2002,2026-01-11,4000,Vendor A,EUR,50.00,2026-01-11T21:04:10.393Z
TXN-2003,2026-01-11,4020,Vendor E,USD,100.00,2026-01-11T21:04:10.393Z


posting_date,account_id,currency,expected_total,source
2026-01-10,4000,EUR,200.00,mock_gl
2026-01-10,4010,EUR,6.00,mock_gl
2026-01-11,4000,EUR,100.00,mock_gl
2026-01-11,4020,USD,100.00,mock_gl
